Day 2 — Build the Portfolio

Today’s goal:
- Define the portfolio tickers
- Assign portfolio weights
- Fetch 15 years of daily price data
- Fetch names from yfinance (sanity check only)
- Save output files to /data

In [1]:
import yfinance as yf
import pandas as pd
import json
from pathlib import Path

In [2]:
portfolio_tickers = [
    "BGRFX",
    "FLPSX",
    "FSCRX",
    "PRFDX",
    "PRGFX",
    "PRNHX",
    "VFIAX",
    "VTMGX",
    "VBILX",
    "VIMAX",
    "VTMSX"
]

In [3]:
# Weights taken directly from user's portfolio allocation

portfolio_weights = {
    "BGRFX": 0.027,
    "FLPSX": 0.068,
    "FSCRX": 0.017,
    "PRFDX": 0.08,
    "PRGFX": 0.058,
    "PRNHX": 0.027,
    "VFIAX": 0.19,
    "VTMGX": 0.119,
    "VBILX": 0.102,
    "VIMAX": 0.093,
    "VTMSX": 0.075
}

# Display sum of weights to verify
total_weight = sum(portfolio_weights.values())
print(f"Total Portfolio Weight: {total_weight}")

# Normalize weights to sum to 1
normalized_weights = {k: v / total_weight for k, v in portfolio_weights.items()}
print("Normalized Weights:", normalized_weights)

total_normalized_weight = sum(normalized_weights.values())
print(f"Total Normalized Portfolio Weight: {total_normalized_weight}")

# Create a normalized portfolio weights dictionary
normalized_portfolio_weights = {
    "BGRFX": 0.027 / total_weight,
    "FLPSX": 0.068 / total_weight,
    "FSCRX": 0.017 / total_weight,
    "PRFDX": 0.08 / total_weight,
    "PRGFX": 0.058 / total_weight,
    "PRNHX": 0.027 / total_weight,
    "VFIAX": 0.19 / total_weight,
    "VTMGX": 0.119 / total_weight,
    "VBILX": 0.102 / total_weight,
    "VIMAX": 0.093 / total_weight,
    "VTMSX": 0.075 / total_weight
}

Total Portfolio Weight: 0.856
Normalized Weights: {'BGRFX': 0.03154205607476636, 'FLPSX': 0.07943925233644861, 'FSCRX': 0.019859813084112152, 'PRFDX': 0.09345794392523366, 'PRGFX': 0.0677570093457944, 'PRNHX': 0.03154205607476636, 'VFIAX': 0.2219626168224299, 'VTMGX': 0.13901869158878505, 'VBILX': 0.1191588785046729, 'VIMAX': 0.10864485981308411, 'VTMSX': 0.08761682242990654}
Total Normalized Portfolio Weight: 1.0


In [4]:
def get_fund_name(ticker):
    try:
        info = yf.Ticker(ticker).info
        return info.get("longName") or info.get("shortName") or ticker
    except:
        return ticker

portfolio_names = {t: get_fund_name(t) for t in portfolio_tickers}
portfolio_names

{'BGRFX': 'Baron Growth Fund',
 'FLPSX': 'Fidelity Low-Priced Stock',
 'FSCRX': 'Fidelity Small Cap Discovery',
 'PRFDX': 'T. Rowe Price Equity Income',
 'PRGFX': 'T. Rowe Price Growth Stock',
 'PRNHX': 'T. Rowe Price New Horizons',
 'VFIAX': 'Vanguard 500 Index Fund',
 'VTMGX': 'Vanguard Developed Markets Index Admiral',
 'VBILX': 'Vanguard Intermediate-Term Bond Index Fund',
 'VIMAX': 'Vanguard Mid Cap Index Admiral',
 'VTMSX': 'Vanguard Tax-Managed Small Cap Adm'}

In [5]:
from datetime import date
from dateutil.relativedelta import relativedelta

def get_price_data(tickers):
    """
    Download 15 years of daily price data (Close) ending today.
    Returns a clean, forward/backward-filled DataFrame with sorted index.
    """
    end = date.today()
    start = end - relativedelta(years=15)

    data = yf.download(
        tickers=tickers,
        start=start,
        end=end,
        progress=False
    )["Close"]

    return data.ffill().bfill().sort_index()

In [6]:
portfolio_prices = get_price_data(portfolio_tickers)

C:\Users\agbro\AppData\Local\Temp\ipykernel_29128\3781159362.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(


In [7]:
display_df = portfolio_prices.copy()
display_df.columns = [f"{t} ({portfolio_names[t]})" for t in portfolio_prices.columns]

display(display_df.head())
display(display_df.tail())
print("Shape:", portfolio_prices.shape)

,BGRFX (Baron Growth Fund),FLPSX (Fidelity Low-Priced Stock),FSCRX (Fidelity Small Cap Discovery),PRFDX (T. Rowe Price Equity Income),PRGFX (T. Rowe Price Growth Stock),PRNHX (T. Rowe Price New Horizons),VBILX (Vanguard Intermediate-Term Bond Index Fund),VFIAX (Vanguard 500 Index Fund),VIMAX (Vanguard Mid Cap Index Admiral),VTMGX (Vanguard Developed Markets Index Admiral),VTMSX (Vanguard Tax-Managed Small Cap Adm)
Date,,,,,,,,,,,
2010-11-15,6.776682,2.456347,1.732953,4.573442,7.667696,2.117864,6.254047,84.126564,69.140511,7.268148,20.846443
2010-11-16,6.705895,2.418568,1.697962,4.509980,7.525657,2.092692,6.286534,82.786888,67.953110,7.099122,20.442045
2010-11-17,6.727565,2.429362,1.708090,4.503837,7.550574,2.103578,6.270295,82.817322,68.271873,7.136682,20.466810
2010-11-18,6.798351,2.461069,1.734794,4.561158,7.697601,2.136914,6.259463,84.088501,69.299934,7.305709,20.821667
2010-11-19,6.833022,2.467816,1.736636,4.567299,7.737472,2.149840,6.264878,84.309242,69.762100,7.305709,20.887701


,BGRFX (Baron Growth Fund),FLPSX (Fidelity Low-Priced Stock),FSCRX (Fidelity Small Cap Discovery),PRFDX (T. Rowe Price Equity Income),PRGFX (T. Rowe Price Growth Stock),PRNHX (T. Rowe Price New Horizons),VBILX (Vanguard Intermediate-Term Bond Index Fund),VFIAX (Vanguard 500 Index Fund),VIMAX (Vanguard Mid Cap Index Admiral),VTMGX (Vanguard Developed Markets Index Admiral),VTMSX (Vanguard Tax-Managed Small Cap Adm)
Date,,,,,,,,,,,
2025-11-10,75.519997,41.040001,24.990000,37.650002,123.970001,58.090000,10.56,631.190002,360.790009,19.700001,97.360001
2025-11-11,76.389999,41.220001,25.020000,38.009998,123.720001,57.930000,10.59,632.500000,361.720001,19.809999,97.690002
2025-11-12,76.760002,41.340000,25.070000,38.189999,123.580002,57.830002,10.59,632.919983,361.779999,19.950001,97.639999
2025-11-13,76.050003,41.009998,24.559999,37.750000,121.059998,56.029999,10.56,622.460022,355.730011,19.719999,95.589996
2025-11-14,75.529999,40.840000,24.580000,37.639999,121.320000,56.200001,10.55,622.280029,355.369995,19.690001,95.489998


Shape: (3774, 11)


In [8]:
data_dir = Path("..") / "data"
data_dir.mkdir(exist_ok=True)

json.dump(portfolio_tickers, open(data_dir/"portfolio_tickers.json","w"), indent=4)
json.dump(normalized_portfolio_weights, open(data_dir/"portfolio_weights.json","w"), indent=4)
json.dump(portfolio_names, open(data_dir/"portfolio_names.json","w"), indent=4)

portfolio_prices.to_csv(data_dir/"portfolio_prices.csv")
portfolio_prices.to_parquet(data_dir/"portfolio_prices.parquet")

print("Day 2 complete: Portfolio artifacts saved to /data")

Day 2 complete: Portfolio artifacts saved to /data
